In [ ]:
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import chromadb
from langchain_chroma import Chroma


from langchain.schema import Document

from langchain.chains.question_answering import load_qa_chain
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv() # Loads .env file

genai.configure(api_key=os.getenv("GOOGLE_API_KEY")) 

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2
)

response = llm.invoke("Welcome to Egypt 🇪🇬")
print(response.content)


In [ ]:
import json

# Open and read the JSON file
with open('docs.json', 'r') as file:
    data = json.load(file)


In [ ]:
documents = []

In [ ]:
for item in data:
    page_content = item["page_content"]
    
    # Flatten metadata and remove None values
    metadata = {
        "document_title": item.get("document_title"),
        "section_title": item.get("section_title"),
    }

    # Merge nested metadata if it exists
    if "metadata" in item and isinstance(item["metadata"], dict):
        for key, value in item["metadata"].items():
            if isinstance(value, dict):
                metadata.update(value)
            else:
                metadata[key] = value

    # Remove keys with None values
    metadata = {k: v for k, v in metadata.items() if v is not None}

    # Create Document
    documents.append(Document(page_content=page_content, metadata=metadata))


In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]


In [ ]:
    # Create embeddings using a Google Generative AI model
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-exp-03-07",google_api_key="**********")


In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [ ]:
batch1 = documents[0:]
batch2 = documents[100:200]
batch3 = documents[200:]

batch1_ids = uuids[0:]
batch2_ids = uuids[100:200]
batch3_ids = uuids[200:]
# 

In [ ]:

batches = [
    (batch1, batch1_ids),
    (batch2, batch2_ids),
    (batch3, batch3_ids)
]


In [ ]:
import time
index = 0

while True:
    docs, ids = batches[index]
    vector_store.add_documents(documents=docs, ids=ids)

    index = (index + 1) % len(batches)
    time.sleep(180)  # Wait for 3 minutes


In [ ]:
from langchain_chroma import Chroma

VDB = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [ ]:
results = VDB.as_retriever(
    k=3,
    
)


In [ ]:
results

In [ ]:
import chromadb

client = chromadb.Client()


client = chromadb.PersistentClient(path="./chroma_langchain_db")

collection_name = "example_collection"
collection = client.get_or_create_collection(name=collection_name)

In [ ]:
results = vector_store.similarity_search(
    "  افضل الاماكن للسياحة في مصر",
    k=2,
    
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
vector_store

In [ ]:
dense_retriever = vector_db.as_retriever(search_kwargs={'k' : 5})


In [ ]:
from langchain.prompts import PromptTemplate , ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0.3)

system_prompt = """
Context: النص التالي يحتوي على معلومات سياحية، تاريخية، وإرشادات سفر متعلقة بمصر بناءً على طلب المستخدم:
{context}

🔍 المهمة:
1) **تحديد نوع الطلب:**
    - إذا كان الإدخال **سؤالاً عن كيفية القيام بشيء** (مثل: "كيف أصل إلى الأهرامات؟" أو "خطوات استخراج تأشيرة")، فاستخرج الخطوات العملية أو الإجراءات.
    - إذا كان الإدخال **طلباً لمعلومات أو وصف** (مثل: "ما هي أهم المعالم في الأقصر؟" أو "ماذا آكل في القاهرة؟")، فاستخرج التفاصيل والترشيحات الأنسب.

2) **تحديد المعلومات ذات الصلة:**
    - حدد **جميع المعلومات المرتبطة** بالطلب من السياق المرفق، واستخدمها كمصدر وحيد للمعلومات (لا تذكر أسماء الملفات أو المصادر أو المراجع في الإجابة).

3) **استخراج المعلومات الأساسية:**
    - استخرج فقط التفاصيل التي تُجيب على طلب المسافر بشكل مباشر وواضح.

قواعد التنسيق:
- إذا كانت الإجابة تتعلق بـ **خطوات، مسارات رحلات، أو إجراءات**:
   - استخدم **قائمة مرقّمة واحدة** (1.، 2.، 3.، ...) لتوضيح التسلسل.
- إذا كانت الإجابة **وصفية، نصائح عامة، أو ترشيحات لأماكن**:
   - استخدم **نقاط تعداد فقط (•)**.

⚠️ **ممنوع تماماً** ذكر أي إشارات للمصدر (مثل: "بناءً على الوثيقة" أو "المصدر رقم 1") أو أي بيانات تقنية من السياق.
لا تخلط بين التنسيقات (إما أرقام أو نقاط).
إذا كان هناك شك في طبيعة الطلب، استخدم القائمة المرقّمة.

**أخرج الإجابة بتنسيق واحد فقط كما هو موضّح أعلاه، بلهجة ودودة ومهنية تناسب خبيراً سياحياً مصرياً**:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system_prompt),
        ('human' , "جاوب علي السؤال ده: {input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model , prompt)
rag_chain = create_retrieval_chain(dense_retriever , question_answer_chain)


In [ ]:
system_prompt = """
Context: النص التالي يحتوي على معلومات سياحية، تاريخية، وإرشادات سفر متعلقة بمصر بناءً على طلب المستخدم:
{context}

🔍 المهمة:
1) **تحديد نوع الطلب:**
    - إذا كان الإدخال **سؤالاً عن كيفية القيام بشيء** (مثل: "كيف أصل إلى الأهرامات؟" أو "خطوات استخراج تأشيرة")، فاستخرج الخطوات العملية أو الإجراءات.
    - إذا كان الإدخال **طلباً لمعلومات أو وصف** (مثل: "ما هي أهم المعالم في الأقصر؟" أو "ماذا آكل في القاهرة؟")، فاستخرج التفاصيل والترشيحات الأنسب.

2) **تحديد المعلومات ذات الصلة:**
    - حدد **جميع المعلومات المرتبطة** بالطلب من السياق المرفق، واستخدمها كمصدر وحيد للمعلومات (لا تذكر أسماء الملفات أو المصادر أو المراجع في الإجابة).

3) **استخراج المعلومات الأساسية:**
    - استخرج فقط التفاصيل التي تُجيب على طلب المسافر بشكل مباشر وواضح.

قواعد التنسيق:
- إذا كانت الإجابة تتعلق بـ **خطوات، مسارات رحلات، أو إجراءات**:
   - استخدم **قائمة مرقّمة واحدة** (1.، 2.، 3.، ...) لتوضيح التسلسل.
- إذا كانت الإجابة **وصفية، نصائح عامة، أو ترشيحات لأماكن**:
   - استخدم **نقاط تعداد فقط (•)**.

⚠️ **ممنوع تماماً** ذكر أي إشارات للمصدر (مثل: "بناءً على الوثيقة" أو "المصدر رقم 1") أو أي بيانات تقنية من السياق.
لا تخلط بين التنسيقات (إما أرقام أو نقاط).
إذا كان هناك شك في طبيعة الطلب، استخدم القائمة المرقّمة.

**أخرج الإجابة بتنسيق واحد فقط كما هو موضّح أعلاه، بلهجة ودودة ومهنية تناسب خبيراً سياحياً مصرياً**:
{context}
"""


In [ ]:
# الاستعلام عن مواعيد الزيارة وأسعار التذاكر (مثال)
dense_retriever.invoke("ما هي مواعيد زيارة منطقة الأهرامات وما هي أسعار التذاكر للأجانب والعرب؟")

In [ ]:
def Answer_question(Question):
    question = str(Question)
    response = rag_chain.invoke({"input": question})
    return response['answer']

In [ ]:
# الاستعلام عن مدة الإقامة وتوقيت تجديد التأشيرة
q = "ما هي مدة الإقامة المسموح بها لتأشيرة السياحة في مصر، ومتى يجب عليّ البدء في إجراءات التجديد؟"

In [ ]:
print(Answer_question(q))


In [ ]:
q2 = "ما هي المدة المسموحة للإقامة السياحية في مصر، ومتى يجب على السائح (سواء كان قادماً بتأشيرة إلكترونية أو عند الوصول) تقديم طلب لتمديد إقامته؟"

In [ ]:
print(Answer_question(q2))